In [44]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import cplex
import optlang
import math
from optlang.symbolics import Zero, add
import cobra.util.solver as sutil
from cobra.flux_analysis.parsimonious import pfba
from IPython.core.display import HTML
from configparser import ConfigParser
config = ConfigParser()
config.read("config.cfg")
paths = config.get("script", "syspaths").split(";")
for path in paths:
    sys.path.append(path)
from modelseedpy import MSPackageManager, MSExpression
from MSExpression import LOWEST

In [47]:
model= cobra.io.read_sbml_model("iML1515.xml")
model.solver = 'optlang-cplex'
medium = {'EX_glc__D_e':10,
         'EX_pro__L_e':0.1,
         'EX_ile__L_e':0.1,
         'EX_met__D_e':0.1,
         'EX_lys__L_e':0.1,
         'EX_26dap__M_e':0.1,
         'EX_thm_e':0.1,
         'EX_so4_e':100,
         'EX_pi_e':100,
         'EX_cit_e':0.1,
         'EX_mn2_e': 100,
         'EX_fe2_e': 100,
         'EX_na1_e': 100,
         'EX_mg2_e': 100,
         'EX_nh4_e': 100,
         'EX_k_e': 100,
         'EX_o2_e': 20,
         'EX_h2o_e': 100,
         'EX_h_e': 100,
         'EX_fe3_e': 100.0,
         'EX_zn2_e': 100.0,
         'EX_ca2_e': 100.0,
         'EX_ni2_e': 100.0,
         'EX_cu2_e': 100.0,
         'EX_sel_e': 100.0,
         'EX_cobalt2_e': 100.0,
         'EX_mobd_e': 100.0,
         'EX_cl_e': 100.0,
         'EX_cbl1_e':100,
         'EX_tungs_e': 100.0,
         'EX_slnt_e': 100.0
}
pkgmgr = MSPackageManager.get_pkg_mgr(model)
pkgmgr.getpkg("ElementUptakePkg").build_package({"C":60})
model.medium=medium
#Fixing fluxes at experimental values
removed_bounds = dict()
removed_bounds["THRS"] = [model.reactions.get_by_id("THRS").lower_bound,model.reactions.get_by_id("THRS").upper_bound]
removed_bounds["BIOMASS_Ec_iML1515_WT_75p37M"] = [model.reactions.get_by_id("BIOMASS_Ec_iML1515_WT_75p37M").lower_bound,model.reactions.get_by_id("BIOMASS_Ec_iML1515_WT_75p37M").upper_bound]
model.reactions.THRS.lower_bound=0.41
model.reactions.THRS.upper_bound=0.41
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.upper_bound=0.33
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.lower_bound=0.33
#Setting objective
biomass_objective = model.problem.Objective(
    1 * model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.flux_expression,
    direction='max')
model.objective = biomass_objective
#Optimizing
sol=model.optimize()
#Loading expression data 
WTExpression = MSExpression.from_gene_feature_file("277_IPTG9h.txt",None,False)
RxnWTExpression = WTExpression.build_reaction_expression(model,LOWEST)
# minimize reactions with low expression
removed_lowex=[]
count = 0
for rxn in wt_rnaseq.index:
    if wt_rnaseq.at[rxn,'fpkm'] < 5:
        #print (rxn, wt_rnaseq.at[rxn,'fpkm'])
        with model:
#             exp=model.reactions.get_by_id(rxn).flux_expression
#             objective = model.problem.Objective(exp, direction='min')
#             model.objective = objective
            model.reactions.get_by_id(rxn).upper_bound=0
            model.reactions.get_by_id(rxn).lower_bound=0

            sol=model.optimize()
            
        #print(rxn, sol.fluxes[rxn], sol.objective_value)
        if sol.fluxes[rxn]==0 and sol.status != 'infeasible':
            removed_lowex.append(rxn)
            removed_bounds[rxn] = [model.reactions.get_by_id(rxn).lower_bound,model.reactions.get_by_id(rxn).upper_bound]
            model.reactions.get_by_id(rxn).upper_bound=0
            model.reactions.get_by_id(rxn).lower_bound=0
        else:
            count += 1
refflux = cobra.flux_analysis.pfba(model)
print(count)
print(len(removed_lowex))
#HTML(refflux.fluxes.to_frame().to_html(table_id='solution1'))
#values at 1: 27 failed; 205 successfully turned off
#at 5: 78 failed; 383 successfully turned off
#MAKE SURE WE'RE USING NORMALIZED EXPRESSION VALUES


In [38]:
# #minimize negative exchange fluxes 

# #bio_var= -100*model.reactions.BIOMASS_Ec_iML1515_core_75p37M.flux_expression
# #const_list=[bio_var]
# const_list=[]
# for rxn in model.reactions:
#     if "EX_" in rxn.id and model.reactions.get_by_id(rxn.id).flux <0:
#         rxn_fluxex= abs(model.reactions.get_by_id(rxn.id).flux_expression)
#         const_list.append(rxn_fluxex)

# const_exp= sum(const_list)
# print (const_exp)

# new_objective = model.problem.Objective(const_exp, direction='min')
# model.objective = new_objective

In [51]:
# minimize reactions with low flux from WT rnaseq
removed_lowex=[]
count = 0
for rxn in wt_rnaseq.index:
    if wt_rnaseq.at[rxn,'fpkm'] < 5:
        #print (rxn, wt_rnaseq.at[rxn,'fpkm'])
        with model:
#             exp=model.reactions.get_by_id(rxn).flux_expression
#             objective = model.problem.Objective(exp, direction='min')
#             model.objective = objective
            model.reactions.get_by_id(rxn).upper_bound=0
            model.reactions.get_by_id(rxn).lower_bound=0

            sol=model.optimize()
            
        #print(rxn, sol.fluxes[rxn], sol.objective_value)
        if sol.fluxes[rxn]==0 and sol.status != 'infeasible':
            removed_lowex.append(rxn)
            removed_bounds[rxn] = [model.reactions.get_by_id(rxn).lower_bound,model.reactions.get_by_id(rxn).upper_bound]
            model.reactions.get_by_id(rxn).upper_bound=0
            model.reactions.get_by_id(rxn).lower_bound=0
        else:
            count += 1
refflux = cobra.flux_analysis.pfba(model)
print(count)
print(len(removed_lowex))
#HTML(refflux.fluxes.to_frame().to_html(table_id='solution1'))
#values at 1: 27 failed; 205 successfully turned off
#at 5: 78 failed; 383 successfully turned off
#MAKE SURE WE'RE USING NORMALIZED EXPRESSION VALUES

78
383


In [24]:
#Removing locks on all reactions with fixed zero flux
for rxn in removed_lowex:
    model.reactions.get_by_id(rxn).lower_bound = removed_bounds[rxn][0]
    model.reactions.get_by_id(rxn).upper_bound = removed_bounds[rxn][1]

#Removing locks on biomass and threonine
model.reactions.get_by_id("THRS").lower_bound = removed_bounds["THRS"][0]
model.reactions.get_by_id("THRS").upper_bound = removed_bounds["THRS"][1]
model.reactions.get_by_id("BIOMASS_Ec_iML1515_WT_75p37M").lower_bound = removed_bounds["BIOMASS_Ec_iML1515_WT_75p37M"][0]
model.reactions.get_by_id("BIOMASS_Ec_iML1515_WT_75p37M").upper_bound = removed_bounds["BIOMASS_Ec_iML1515_WT_75p37M"][1]

#Loading modified expression profile
fc1= pd.read_csv('643_IPTG9h_FC.txt',sep='\t', header='infer',index_col=0,)
fc_df = fc1.groupby('rxn').mean()
wtfc = dict()
coef = dict()

#Computing fluxes to fit
rxns=list(refflux.fluxes.index)
for rxn in rxns:
    if rxn not in fc_df.index:
        if abs(refflux.fluxes[rxn]) > 1e-7:
            wtfc[rxn]=refflux.fluxes[rxn]
        else:
            wtfc[rxn]=0
    elif fc_df.loc[rxn,'fc'] > 1:
        #print(rxn+"\t"+str(fc_df.loc[rxn,'fc']))
        #These are reactions that had larger flux in the second study
        if abs(refflux.fluxes[rxn]) > 1e-7:
            wtfc[rxn]=fc_df.loc[rxn,'fc']*refflux.fluxes[rxn]
        else:
            wtfc[rxn]=0.001
    else:
        #print(rxn+"\t"+str(fc_df.loc[rxn,'fc']))
        wtfc[rxn]=fc_df.loc[rxn,'fc']*refflux.fluxes[rxn]
    if wtfc[rxn] > 1000:
        wtfc[rxn] = 1000
    coef[rxn] = abs(wtfc[rxn]-refflux.fluxes[rxn])**0.07
    if coef[rxn] < 1:
        coef[rxn] = 1
    #print(rxn+"\t"+str(wtfc[rxn]))
print(str(coef["THRS"]))

1.5836241252675023


In [27]:
#Fitting flux to fold change flux
with model:
    prob = model.problem
    model.objective = prob.Objective(Zero, direction="min", sloppy=True)
    to_add = []
    obj_vars = []
    obj_coef = dict()
    linear = 1
    for r in model.reactions:
        flux = wtfc[r.id]
        if linear:
            components = sutil.add_absolute_expression(
                model, r.flux_expression, name="moma_dist_" + r.id,
                difference=flux, add=False)
            to_add.extend(components)
            obj_vars.append(components.variable)
            obj_coef[components.variable] = coef[r.id]
        else:
            dist = prob.Variable("moma_dist_" + r.id)
            const = prob.Constraint(r.flux_expression - dist, lb=flux, ub=flux,
                                    name="moma_constraint_" + r.id)
            to_add.extend([dist, const])
            obj_vars.append(dist ** 2)
    model.add_cons_vars(to_add)
    if linear:
        #model.objective.set_linear_coefficients({v: 1.0 for v in obj_vars})
        model.objective.set_linear_coefficients(obj_coef)
    else:
        model.objective = prob.Objective(
            add(obj_vars), direction="min", sloppy=True)
    wtfc_solution = model.optimize()
print(wtfc_solution.fluxes.THRS)
#HTML(wtfc_solution.fluxes.to_frame().to_html(table_id='solution2'))

0.6758431772567377


In [31]:
#Now I'm going to fix the threonine flux at 1.5 times the flux from the fold change analysis
#Then I will repeat linear MOMA to find the minimal reactions that must change to reach this new production
#microtitre plate concerns: ODs will be 1/3 of ODs in RNA-seq experiments;
with model:
    model.reactions.THRS.lower_bound = 1.5*wtfc_solution.fluxes.THRS
    model.reactions.THRS.upper_bound = 1.5*wtfc_solution.fluxes.THRS
    prob = model.problem
    model.objective = prob.Objective(Zero, direction="min", sloppy=True)
    to_add = []
    obj_vars = []
    obj_coef = dict()
    for r in model.reactions:
        flux = wtfc_solution[r.id]
        components = sutil.add_absolute_expression(
            model, r.flux_expression, name="moma_dist_" + r.id,
            difference=flux, add=False)
        to_add.extend(components)
        obj_vars.append(components.variable)
        obj_coef[components.variable] = 1
    model.add_cons_vars(to_add)
    model.objective.set_linear_coefficients(obj_coef)
    goal_solution = model.optimize()
HTML(goal_solution.fluxes.to_frame().to_html(table_id='solution3'))

,fluxes
CYTDK2,0.000000e+00
XPPT,0.000000e+00
HXPRT,0.000000e+00
NDPK5,0.000000e+00
SHK3Dr,8.409328e-02
NDPK6,0.000000e+00
NDPK8,4.916414e-03
DHORTS,-7.246980e-02
OMPDC,7.246980e-02
PYNP2r,0.000000e+00


In [ ]:
#ALL CODE CELLS BELOW THIS POINT HAVENT BEEN MODIFIED FROM ORIGINAL NOTEBOOK

In [23]:
sol2_list=[]
for rxn in sol2_fitFC.fluxes.index:
    if abs(sol2_fitFC.fluxes[rxn])>0:
        if rxn in fc_df.index:
            fc_numb= fc_df.at[rxn,'fc']
        else:
            fc_numb='N/A'
        line=[rxn,sol1_minlow.fluxes[rxn],fc_numb,wtfc[rxn],sol2_fitFC.fluxes[rxn] ]
        sol2_list.append(line)
        
sol2_df= pd.DataFrame(sol2_list, columns=['rxn','fit_WT','FC','WTxFC','fit_WTxFC'])
sol2_df.to_csv("sol2_df.txt")
sol2_df

,rxn,fit_WT,FC,WTxFC,fit_WTxFC
0,NDPK5,0.043453,0,0.043453,0.036263
1,SHK3Dr,0.616114,0.47965,0.295519,0.528120
2,NDPK8,-44.554291,N/A,-44.554291,-44.501148
3,DHORTS,-0.534254,0,-0.534254,-0.534254
4,OMPDC,0.534254,0,0.534254,0.534254
...,...,...,...,...,...
794,COLIPAabctex,0.002690,N/A,0.002690,0.002206
795,ENLIPAabctex,0.000000,N/A,0.000000,0.011097
796,DM_mththf_c,0.001023,N/A,0.001023,0.001023
797,MPTS,0.000005,N/A,0.000005,0.000004


In [24]:
sol1_minlow.fluxes.to_csv('sol1.txt')
sol2_fitFC.fluxes.to_csv('sol2.txt')

In [35]:
change={}
for rxn in sol2_fitFC.fluxes.index:
    if rxn in sol1_minlow.fluxes.index:
        if abs(sol1_minlow.fluxes[rxn])==0:
            sol1_approx=0.001
        else:
            sol1_approx= sol1_minlow.fluxes[rxn]
        calc=sol2_fitFC.fluxes[rxn]/sol1_approx
        if calc <0.00001:
            calc=0
        if calc >100:
            calc=100
            
        try:
            #calc2=math.log(calc,2)
            change[rxn]= calc
        except:
            pass

import csv
with open('sol2_vs_sol1.csv', 'w') as f:
    for key in change.keys():
        f.write("%s,%s\n"%(key,change[key]))
        

In [26]:
# #print only reactions with fC data 
# change={}
# for rxn in sol2_fitFC.fluxes.index:
#     if rxn in fc_df.index and abs(wtfc[rxn])>0 and abs(sol2_fitFC.fluxes[rxn]) >0:
#         fc=sol2_fitFC.fluxes[rxn]/wtfc[rxn]
       
#         try:
#             calc=math.log(fc,2)
#             change[rxn]= calc
#             #print(rxn,sol2_fitFC.fluxes[rxn],wtfc[rxn], calc)
#         except:
#             pass

# import csv
# with open('sol2_vs_wtfc.csv', 'w') as f:
#     for key in change.keys():
#         f.write("%s,%s\n"%(key,change[key]))

In [27]:
sol2_fitFC.fluxes['THRS']

0.34115606811849897

In [28]:
sol2_fitFC.fluxes["BIOMASS_Ec_iML1515_WT_75p37M"]

0.2706286072850254

In [29]:
model.reactions.THRS.lower_bound = 1.68
model.reactions.THRS.upper_bound = 1.68

model.reactions.THRS

Reaction identifier,THRS
Name,Threonine synthase
Memory address,0x07f7eff337c10
Stoichiometry,h2o_c + phom_c --> pi_c + thr__L_c H2O H2O + O-Phospho-L-homoserine --> Phosphate + L-Threonine
GPR,b0004
Lower bound,1.68
Upper bound,1.68


In [30]:
sol3= moma(model,sol2_fitFC)

print (sol3.fluxes['THRS'])
print (sol3.fluxes["BIOMASS_Ec_iML1515_WT_75p37M"])

1.68
0.2520412497946419


In [31]:
for rxn in sol3.fluxes.index:
    if abs(sol3.fluxes[rxn])>0:
        if rxn in fc_df.index:
            fc_numb= fc_df.at[rxn,'fc']
        else:
            fc_numb='N/A'
    print (rxn,sol1_minlow.fluxes[rxn],fc_numb,wtfc[rxn],sol2_fitFC.fluxes[rxn], sol3.fluxes[rxn] )

CYTDK2 0.0 N/A 0.0 0.0 0.0
XPPT 0.0 N/A 0.0 0.0 0.0
HXPRT 0.0 N/A 0.0 0.0 0.0
NDPK5 0.04345264447078663 0.0 0.04345264447078663 0.03626330052575867 0.03578724112571497
SHK3Dr 0.6161143674277071 0.4796501385 0.2955193416685396 0.528120474959592 0.528120474959592
NDPK6 0.0 0.4796501385 0.0 0.0 0.0
NDPK8 -44.55429126423425 N/A -44.55429126423425 -44.50114807116259 -44.50114807116259
DHORTS -0.5342538134197303 0.0 -0.5342538134197303 -0.5342538134197303 -0.5342538134197303
OMPDC 0.5342538134197303 0.0 0.5342538134197303 0.5342538134197303 0.5342538134197303
PYNP2r 0.0 0.13477925400000001 0.000134779254 0.000134779254 0.00013477925399999998
G5SD 0.0 0.13477925400000001 0.0 0.0 0.0
CS 42.74129826900181 1.285597766 54.94811757056839 42.74129826900172 42.74129826900172
ICDHyr 16.170085711367697 N/A 16.170085711367697 16.285553241486742 16.285553241486742
ALATA_L2 0.0 N/A 0.0 0.0 0.0
DURIPP 0.0 0.21370730300000002 0.00021370730300000003 0.00021370730300000003 0.00021370730300000003
ACALD 44.554

In [35]:
 
change={}
for rxn in sol3.fluxes.index:
    if rxn in sol2_fitFC.fluxes.index:
        if sol2_fitFC.fluxes[rxn] ==0:
            sol2_approx=0.001
        else:
            sol2_approx= sol2_fitFC.fluxes[rxn]
        fc=sol3.fluxes[rxn]/sol2_approx
       
        #print (sol3.fluxes[rxn],sol2_fitFC.fluxes[rxn] )
        try:
            calc=math.log(fc,2)
            change[rxn]= calc
        except:
            pass
  
with open('sol3_vs_sol2.csv', 'w') as f:
    for key in change.keys():
        f.write("%s,%s\n"%(key,change[key]))

In [53]:
# #print only reactions with fC data 
# change={}
# for rxn in sol3.fluxes.index:
#     if rxn in wtfc.keys() and rxn in fc_df.index:
#         if abs(wtfc[rxn]) >0:
#             fc=sol3.fluxes[rxn]/wtfc[rxn]

#             try:
#                 calc=math.log(fc,2)
#                 change[rxn]= calc
#                 #print(rxn,sol2_fitFC.fluxes[rxn],wtfc[rxn], calc)
#             except:
#                 pass

# import csv
# with open('sol3_vs_WTxfc.csv', 'w') as f:
#     for key in change.keys():
#         f.write("%s,%s\n"%(key,change[key]))

In [31]:
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.upper_bound=1.38
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.lower_bound=1.38

model.reactions.BIOMASS_Ec_iML1515_WT_75p37M

Reaction identifier,BIOMASS_Ec_iML1515_WT_75p37M
Name,E. coli biomass objective function (iML1515) - WT - with 75.37 GAM estimate
Memory address,0x07f911cf28290
Stoichiometry,0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala__L_c + 0.000223 amet_c + 0.28742 arg__L_c + 0.23... 0.000223 10-Formyltetrahydrofolate + 0.000223 2-Demethylmenaquinol 8 + 2.5e-05 [2Fe-2S] iron-sulfur cluster + 0.000248 [4Fe-4S] iron-sulfur cluster + 0.000223 5-Methyltetrahydrofolate + 0.000279 Ac...
GPR,
Lower bound,1.38
Upper bound,1.38


In [32]:
sol4= moma(model,sol3)

print (sol4.fluxes['THRS'])
print (sol4.fluxes["BIOMASS_Ec_iML1515_WT_75p37M"])

1.68
1.38


In [35]:

change={}
for rxn in sol4.fluxes.index:
    if rxn in  sol2_fitFC.fluxes.index:
        if sol2_fitFC.fluxes[rxn] ==0:
            sol2_approx=0.001
        else:
            sol2_approx= sol2_fitFC.fluxes[rxn]
        fc=sol4.fluxes[rxn]/sol2_approx
       
        #print (sol3.fluxes[rxn],sol2_fitFC.fluxes[rxn] )
        try:
            calc=math.log(fc,2)
            change[rxn]= calc
        except:
            pass
  
with open('sol4_vs_sol2.csv', 'w') as f:
    for key in change.keys():
        f.write("%s,%s\n"%(key,change[key]))

In [34]:
for rxn in sol4.fluxes.index:
    if abs(sol4.fluxes[rxn])>0:
        if rxn in fc_df.index:
            fc_numb= fc_df.at[rxn,'fc']
            
        else:
            fc_numb='N/A'
        print (rxn,sol1.fluxes[rxn],fc_numb,wtfc[rxn],sol2_fitFC.fluxes[rxn], sol3.fluxes[rxn], sol4.fluxes[rxn] )
   

NDPK5 0.0435404 0.0 0.0 0.06693316753695357 0.06587855133668898 0.03534456
SHK3Dr 0.6318916999999997 0.4796501385 0.30308694142200165 0.35940386100863614 0.35940386100863614 0.5129473800000086
NDPK8 0.0 N/A 0.0 0.0 0.0 0.011267920794683323
DHORTS -0.5445507999999997 0.0 -0.0 -0.3097265876974513 -0.3097265876974513 -0.44204712
OMPDC 0.5445507999999997 0.0 0.0 0.3097265876974513 0.3097265876974513 0.44204712
PYNP2r 0.0 0.13477925400000001 0.000134779254 0.000134779254 0.00013477925399999965 0.02759897517421374
CS 6.592128836363712 1.285597766 8.474826105213472 10.015306749527161 10.015306749527161 10.015306749527161
ICDHyr 10.492128836363712 N/A 10.492128836363793 9.189543653084346 9.189543653084346 9.189543653084346
DURIPP -0.0421685 0.21370730300000002 -0.009011716406555494 -0.02398436585405802 -0.02398436585405802 -0.02759897517421374
PTRCTA 10.362653036363714 0.0 0.0 6.019143517382363 6.020791159606219 6.020791159606219
PPA 7.6352689181818585 5.7856885455 44.17528792173696 44.1752879